In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [2]:
import sys
sys.path.append('../../../')

from random_search_tensorflow import RandomSearchCNN
import tensorflow as tf

In [3]:
processedDataX_path = "../../preprocessedSamples_cut_500_X_samples_allGuitar_20480_Mm7_R1D.data"
processedDatay_path = "../../preprocessedSamples_cut_500_y_samples_allGuitar_20480_Mm7_R1D.data"
processedData_path = ""

if os.path.isfile(processedDataX_path): #if already preprocessed
    processedX = np.load(processedDataX_path)
    processedy = np.load(processedDatay_path)
else:
    import pandas as pd
    import librosa
    df = pd.read_csv('../../samples_allGuitar_20480_Mm7_R1D.csv')

    X_load = np.array(df.iloc[:,:-1], dtype=np.float)
    y_load = np.array(df.iloc[:,-1], dtype=np.float)
    processedX = np.zeros((len(X_load),12,80,1), dtype=np.float)
    processedy = np.zeros(len(y_load), dtype=np.float)
    X_fft_new = np.zeros(20480)
    for i in range(len(X_load)):
        sample = np.fft.rfft(X_load[i])
        for ii in range(len(sample)):
            if ii < 1000: #ignore frequencies greater than 2kHz
                X_fft_new[ii] = sample[ii]
                
        sample = np.fft.ifft(X_fft_new)
        sample = librosa.feature.chroma_stft(y=sample, sr=44100, n_fft=20480, hop_length=258)
        sample = np.atleast_3d(sample)
        processedX[i] = sample
        processedy[i] = y_load[i]
        if i % 400 == 0:
            print(i)

    processedX.dump(processedDataX_path)
    processedy.dump(processedDatay_path)

from sklearn.utils import shuffle
print(processedy)
sprocessedX, sprocessedy = shuffle(processedX, processedy)
print(len(sprocessedX))

trainRange = int(len(sprocessedX) * 0.8)
validRange = int(len(sprocessedX) * 0.9)
testRange = int(len(sprocessedX) * 0.1)


X_train = np.array(sprocessedX[:trainRange], dtype=np.float)
y_train = np.array(sprocessedy[:trainRange], dtype=np.float)

X_valid = np.array(sprocessedX[trainRange:validRange], dtype=np.float)
y_valid = np.array(sprocessedy[trainRange:validRange], dtype=np.float)

X_test = np.array(sprocessedX[testRange:], dtype=np.float)
y_test = np.array(sprocessedy[testRange:], dtype=np.float)
print(y_test[1])
print(sprocessedX.shape)

[  0.   0.   0. ...,  45.  45.  45.]
3214
47.0
(3214, 12, 80, 1)


In [4]:
import tensorflow as tf

def leaky_relu(alpha=0.01):
		def parametrized_leaky_relu(z, name=None):
			return tf.maximum(alpha * z, z, name=name)
		return parametrized_leaky_relu
    
"""
Best parameters (kfold = 5):

{'n_hidden_layers': 2, 'n_neurons': 500, 'optimizer_class': <class 'tensorflow.python.training.adam.AdamOptimizer'>, 
'learning_rate': 0.05, 'batch_size': 400, 
'activation': <function leaky_relu.<locals>.parametrized_leaky_relu at 0x00000203351C67B8>, 
'dropout_rate': 0.1, 'conv1': {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 
'conv1_dropout': 0.3, 'conv1_activation': <function relu at 0x00000203335AC620>}, 'conv2': {'conv2_fmaps': 16, 
'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.2, 'conv2_activation': <function relu at 0x00000203335AC620>}, 
'architecture': 4, 'accuracy_rate': 0.9958441495895386}


Best parameters (kfold = 1 and cutting freq = 1000):

Best parameters:
{'n_hidden_layers': 8, 'n_neurons': 500, 'optimizer_class': <class 'tensorflow.python.training.adam.AdamOptimizer'>,
'learning_rate': 0.025, 'batch_size': 600,
'activation': <function leaky_relu.<locals>.parametrized_leaky_relu at 0x000001FDB8747950>,
'dropout_rate': 1e-06, 
'conv1': {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 'conv1_dropout': 0.3, 'conv1_activation': <function relu at 0x000001FDA77410D0>}, 
'conv2': {'conv2_fmaps': 16, 'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.3, 'conv2_activation': <function relu at 0x000001FDA77410D0>}, 'architecture': 1, 'accuracy_rate': 0.98441553115844727}

Best parameters (kfold = 5 and cutting freq = 1000):

{'n_hidden_layers': 2, 'n_neurons': 150, 'optimizer_class': <class 'tensorflow.python.training.adagrad.AdagradOptimizer'>, 'learning_rate': 0.025, 'batch_size': 300,
'activation': <function leaky_relu.<locals>.parametrized_leaky_relu at 0x00000213B22AFD90>,
'dropout_rate': 0.2, 
'conv1': {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 'conv1_dropout': 0.2, 'conv1_activation': <function relu at 0x00000213B6A4FA60>},
'conv2': {'conv2_fmaps': 16, 'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.3, 'conv2_activation': <function relu at 0x00000213B6A4FA60>}, 'architecture': 4, 'accuracy_rate': 0.97958478927612302}

Best parameters (kfold = 5 and cutting freq = 500):
{'n_hidden_layers': 1, 'n_neurons': 300, 'optimizer_class': <class 'tensorflow.python.training.adam.AdamOptimizer'>,
'learning_rate': 0.05, 'batch_size': 600, 
'activation': <function leaky_relu.<locals>.parametrized_leaky_relu at 0x000001D24838D378>, 'dropout_rate': 1e-06, 
'conv1': {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 'conv1_dropout': 0.3, 'conv1_activation': <function relu at 0x000001D24678D950>}, 
'conv2': {'conv2_fmaps': 16, 'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.2, 'conv2_activation': <function relu at 0x000001D24678D950>}, 'architecture': 1, 'accuracy_rate': 0.9822956919670105}

Best parameters (kfold = 1 and cutting freq = 500):


"""

params = {'n_hidden_layers': [1,2,3,4,5,6,7,8,9,10],
	'n_neurons' : [50, 150, 250, 300, 350, 400, 450, 500],
	'optimizer_class' : [tf.train.AdamOptimizer, tf.train.AdagradOptimizer],
	'learning_rate' : [0.01, 0.05, 0.1, 0.025],
	'batch_size' : [200, 250, 300, 400, 600],
	'activation' : [leaky_relu(), tf.nn.elu],
	'dropout_rate' : [0.000001, 0.1, 0.2, 0.3, 0.4],
	'conv1' : [{'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.1, 'conv1_activation':tf.nn.elu},
               {'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.2, 'conv1_activation':tf.nn.relu},
               {'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.3, 'conv1_activation':tf.nn.relu},
               {'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.1, 'conv1_activation':tf.nn.elu}],
	'conv2' : [{'conv2_fmaps':16, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.1, 'conv2_activation':tf.nn.elu},
               {'conv2_fmaps':16, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.2, 'conv2_activation':tf.nn.relu},
               {'conv2_fmaps':16, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.3, 'conv2_activation':tf.nn.relu},
               {'conv2_fmaps':16, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.1, 'conv2_activation':tf.nn.elu}],
    'architecture' : [1, 2, 3, 4]
}

rnd_search = RandomSearchCNN(params, k_fold=1, num_random_combinations=300)
rnd_search.fit(X_train, y_train, X_train, y_train, X_valid=X_valid, y_valid=y_valid)

testing 300 combinations.
Trining CNN with parameters: n_hidden_layers: 8, n_neurons: 50, optimizer_class: <class 'tensorflow.python.training.adam.AdamOptimizer'>, learning_rate: 0, dropout_rate: 0, batch_size: 300, activation: <function leaky_relu.<locals>.parametrized_leaky_relu at 0x000001BCD0131EA0>, conv1: {'conv1_fmaps': 16, 'conv1_ksize': 5, 'conv1_stride': 1, 'conv1_dropout': 0.3, 'conv1_activation': <function relu at 0x000001BCCE530950>}, conv2: {'conv2_fmaps': 16, 'conv2_ksize': 5, 'conv2_stride': 1, 'conv2_dropout': 0.3, 'conv2_activation': <function relu at 0x000001BCCE530950>} .architecture: 4, 


../../..\random_search_tensorflow.py:48: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if X_test == None or y_test == None:


***********
(2571, 12, 80, 1)
(2571,)
(321, 12, 80, 1)
(321,)
Epoch 0, train accuracy: 4.3614%, valid. accuracy: 4.0498%, valid. best loss: 3.901845
Epoch 1, train accuracy: 7.1651%, valid. accuracy: 7.4766%, valid. best loss: 3.812066
Epoch 2, train accuracy: 13.7072%, valid. accuracy: 12.7726%, valid. best loss: 3.679372
Epoch 3, train accuracy: 16.1994%, valid. accuracy: 15.2648%, valid. best loss: 3.478811
Epoch 4, train accuracy: 23.0530%, valid. accuracy: 19.9377%, valid. best loss: 3.297911
Epoch 5, train accuracy: 24.6106%, valid. accuracy: 26.7913%, valid. best loss: 3.019340
Epoch 6, train accuracy: 35.8255%, valid. accuracy: 33.3333%, valid. best loss: 2.744611
Epoch 7, train accuracy: 43.3022%, valid. accuracy: 40.4984%, valid. best loss: 2.433115
Epoch 8, train accuracy: 49.5327%, valid. accuracy: 41.7445%, valid. best loss: 2.114383
Epoch 9, train accuracy: 46.7290%, valid. accuracy: 44.2368%, valid. best loss: 2.069331
Epoch 10, train accuracy: 57.9439%, valid. accuracy:

***********
(2571, 12, 80, 1)
(2571,)
(321, 12, 80, 1)
(321,)
Epoch 0, train accuracy: 36.4486%, valid. accuracy: 35.2025%, valid. best loss: 3.711800
Epoch 1, train accuracy: 50.4673%, valid. accuracy: 49.5327%, valid. best loss: 2.373659
Epoch 2, train accuracy: 63.5514%, valid. accuracy: 57.6324%, valid. best loss: 1.580726
Epoch 3, train accuracy: 68.2243%, valid. accuracy: 66.3551%, valid. best loss: 1.369748
Epoch 4, train accuracy: 74.7664%, valid. accuracy: 68.5358%, valid. best loss: 1.123911
Epoch 5, train accuracy: 75.2336%, valid. accuracy: 71.6511%, valid. best loss: 1.020218
Epoch 6, train accuracy: 80.3738%, valid. accuracy: 70.7165%, valid. best loss: 0.883070
Epoch 7, train accuracy: 82.7103%, valid. accuracy: 74.1433%, valid. best loss: 0.883070
Epoch 8, train accuracy: 83.4112%, valid. accuracy: 73.8318%, valid. best loss: 0.755953
Epoch 9, train accuracy: 84.8131%, valid. accuracy: 78.1932%, valid. best loss: 0.748241
Epoch 10, train accuracy: 85.5140%, valid. accur

Epoch 12, train accuracy: 90.9657%, valid. accuracy: 79.4393%, valid. best loss: 0.613228
Epoch 13, train accuracy: 92.6791%, valid. accuracy: 81.6199%, valid. best loss: 0.604884
Epoch 14, train accuracy: 94.7040%, valid. accuracy: 82.5545%, valid. best loss: 0.604884
Epoch 15, train accuracy: 93.4579%, valid. accuracy: 80.9969%, valid. best loss: 0.568339
Epoch 16, train accuracy: 95.4829%, valid. accuracy: 82.2430%, valid. best loss: 0.568339
Epoch 17, train accuracy: 95.4829%, valid. accuracy: 83.8006%, valid. best loss: 0.568339
Epoch 18, train accuracy: 95.6386%, valid. accuracy: 81.6199%, valid. best loss: 0.564387
Epoch 19, train accuracy: 97.1963%, valid. accuracy: 84.7352%, valid. best loss: 0.564387
Epoch 20, train accuracy: 96.1059%, valid. accuracy: 85.6698%, valid. best loss: 0.564387
Epoch 21, train accuracy: 97.0405%, valid. accuracy: 84.7352%, valid. best loss: 0.564387
Epoch 22, train accuracy: 97.3520%, valid. accuracy: 85.6698%, valid. best loss: 0.564387
Epoch 23, 

...

Epoch 22, train accuracy: 95.6386%, valid. accuracy: 82.2430%, valid. best loss: 0.568608
Epoch 23, train accuracy: 94.7040%, valid. accuracy: 80.9969%, valid. best loss: 0.568608
Epoch 24, train accuracy: 94.7040%, valid. accuracy: 81.3084%, valid. best loss: 0.568608
Epoch 25, train accuracy: 95.6386%, valid. accuracy: 82.5545%, valid. best loss: 0.568608
Epoch 26, train accuracy: 96.1059%, valid. accuracy: 80.3738%, valid. best loss: 0.568608
Epoch 27, train accuracy: 95.6386%, valid. accuracy: 83.8006%, valid. best loss: 0.568608
Epoch 28, train accuracy: 96.8847%, valid. accuracy: 83.8006%, valid. best loss: 0.568608
Epoch 29, train accuracy: 97.3520%, valid. accuracy: 82.8660%, valid. best loss: 0.568608
Epoch 30, train accuracy: 95.6386%, valid. accuracy: 83.4891%, valid. best loss: 0.568608
Epoch 31, train accuracy: 94.8598%, valid. accuracy: 81.9315%, valid. best loss: 0.568608
Epoch 32, train accuracy: 97.8193%, valid. accuracy: 84.1121%, valid. best loss: 0.568608
Epoch 33, 

Epoch 46, train accuracy: 97.1962%, valid. accuracy: 85.3583%, valid. best loss: 0.522479
Epoch 47, train accuracy: 96.7290%, valid. accuracy: 81.6199%, valid. best loss: 0.522479
Epoch 48, train accuracy: 97.1963%, valid. accuracy: 83.1776%, valid. best loss: 0.522479
Epoch 49, train accuracy: 98.5981%, valid. accuracy: 84.1121%, valid. best loss: 0.522479
Epoch 50, train accuracy: 93.9252%, valid. accuracy: 85.0467%, valid. best loss: 0.522479
Epoch 51, train accuracy: 97.1963%, valid. accuracy: 85.3583%, valid. best loss: 0.522479
Epoch 52, train accuracy: 97.6635%, valid. accuracy: 82.2430%, valid. best loss: 0.522479
Epoch 53, train accuracy: 95.7944%, valid. accuracy: 84.7352%, valid. best loss: 0.522479
Epoch 54, train accuracy: 95.7944%, valid. accuracy: 84.7352%, valid. best loss: 0.522479
Epoch 55, train accuracy: 98.1308%, valid. accuracy: 84.4237%, valid. best loss: 0.522479
Epoch 56, train accuracy: 98.5981%, valid. accuracy: 85.6698%, valid. best loss: 0.522479
Epoch 57, 

Epoch 52, train accuracy: 97.1963%, valid. accuracy: 86.9159%, valid. best loss: 0.523427
Epoch 53, train accuracy: 94.3925%, valid. accuracy: 85.0467%, valid. best loss: 0.523427
Epoch 54, train accuracy: 96.2617%, valid. accuracy: 85.9813%, valid. best loss: 0.523427
Epoch 55, train accuracy: 96.2617%, valid. accuracy: 85.0467%, valid. best loss: 0.523427
Epoch 56, train accuracy: 95.6386%, valid. accuracy: 85.3583%, valid. best loss: 0.523427
Epoch 57, train accuracy: 98.1308%, valid. accuracy: 85.3583%, valid. best loss: 0.523427
Epoch 58, train accuracy: 96.2617%, valid. accuracy: 83.1776%, valid. best loss: 0.523427
Epoch 59, train accuracy: 98.4424%, valid. accuracy: 84.1122%, valid. best loss: 0.523427
Epoch 60, train accuracy: 97.5078%, valid. accuracy: 84.7352%, valid. best loss: 0.523427
Epoch 61, train accuracy: 97.1963%, valid. accuracy: 86.2928%, valid. best loss: 0.523427
Epoch 62, train accuracy: 97.5078%, valid. accuracy: 86.6044%, valid. best loss: 0.523427
Epoch 63, 

Epoch 44, train accuracy: 98.4424%, valid. accuracy: 83.4891%, valid. best loss: 0.569216
Epoch 45, train accuracy: 98.7539%, valid. accuracy: 85.9813%, valid. best loss: 0.569216
Epoch 46, train accuracy: 98.4424%, valid. accuracy: 80.9969%, valid. best loss: 0.569216
Epoch 47, train accuracy: 98.4424%, valid. accuracy: 82.5545%, valid. best loss: 0.569216
Epoch 48, train accuracy: 98.1308%, valid. accuracy: 83.4891%, valid. best loss: 0.569216
Epoch 49, train accuracy: 99.0654%, valid. accuracy: 84.1121%, valid. best loss: 0.569216
Epoch 50, train accuracy: 99.3769%, valid. accuracy: 82.8660%, valid. best loss: 0.569216
Epoch 51, train accuracy: 99.3769%, valid. accuracy: 83.4891%, valid. best loss: 0.569216
Epoch 52, train accuracy: 97.8193%, valid. accuracy: 85.3583%, valid. best loss: 0.569216
Epoch 53, train accuracy: 99.3769%, valid. accuracy: 84.1121%, valid. best loss: 0.569216
Epoch 54, train accuracy: 99.3769%, valid. accuracy: 81.9315%, valid. best loss: 0.569216
Epoch 55, 

***********
(2571, 12, 80, 1)
(2571,)
(321, 12, 80, 1)
(321,)
Epoch 0, train accuracy: 42.3676%, valid. accuracy: 42.0561%, valid. best loss: 3.623575
Epoch 1, train accuracy: 61.9938%, valid. accuracy: 58.5670%, valid. best loss: 1.837384
Epoch 2, train accuracy: 72.5857%, valid. accuracy: 62.6168%, valid. best loss: 1.306388
Epoch 3, train accuracy: 72.8972%, valid. accuracy: 68.5358%, valid. best loss: 1.117018
Epoch 4, train accuracy: 77.5701%, valid. accuracy: 70.0935%, valid. best loss: 0.957125
Epoch 5, train accuracy: 79.4393%, valid. accuracy: 73.5202%, valid. best loss: 0.885828
Epoch 6, train accuracy: 84.1121%, valid. accuracy: 73.8318%, valid. best loss: 0.820238
Epoch 7, train accuracy: 83.4891%, valid. accuracy: 77.5701%, valid. best loss: 0.754415
Epoch 8, train accuracy: 85.9813%, valid. accuracy: 73.8318%, valid. best loss: 0.685647
Epoch 9, train accuracy: 87.8505%, valid. accuracy: 77.2586%, valid. best loss: 0.685647
Epoch 10, train accuracy: 88.7850%, valid. accur

In [ ]:
def leaky_relu(alpha=0.01):
		def parametrized_leaky_relu(z, name=None):
			return tf.maximum(alpha * z, z, name=name)
		return parametrized_leaky_relu

cnn = CNNClassifier(batch_size=400, learning_rate=0.05, n_hidden_layers=2, n_neurons=500, 
                    optimizer_class=tf.train.AdamOptimizer, activation=leaky_relu(), dropout_rate = 0.1,
                    conv1={'conv1_fmaps':16, 'conv1_ksize':5, 'conv1_stride':1, 'conv1_dropout':0.3, 'conv1_activation':tf.nn.relu},
                    conv2={'conv2_fmaps':16, 'conv2_ksize':5, 'conv2_stride':1, 'conv2_dropout':0.2, 'conv2_activation':tf.nn.relu})

cnn.fit(X_train=X_train, y_train=y_train, X_valid=X_valid, y_valid=y_valid)

score = cnn.accuracy_score(X_test, y_test)
score